In [2]:

!nvidia-smi

Mon Nov 27 22:54:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install --upgrade  textblob gensim pytorch-nlp swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=3f0fbb5351fbd9e0239adbc694a6840423a8d7404835afe5153e078fefc8be26
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import itertools
import sys
from textblob import TextBlob, Word
import numpy as np
import random
import re
import swifter
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import os
import pandas as pd
import gensim
import warnings
import nltk

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
%%writefile get_data.sh
if [ ! -f train_corpus_descriptions_airbnb.csv ]; then
  wget -O train_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
fi

if [ ! -f test_corpus_descriptions_airbnb.csv ]; then
    wget -O test_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/a29bbkg8hi4q4f4/test_corpus_descriptions_airbnb.csv?dl=0
fi

Writing get_data.sh


In [6]:
!bash get_data.sh

--2023-11-27 22:54:53--  https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv [following]
--2023-11-27 22:54:54--  https://www.dropbox.com/s/raw/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd9707fc04aedc20b1343e1cc89.dl.dropboxusercontent.com/cd/0/inline/CIWkZzcNnFUKXAONc6AUfdTfxFOScPE5iCSfgRahkqpvnUOsgPyxgao9IvLIqLyEBML4Ti2dNFRWx-zKkDCGf8nwDXmcxNjFSNI_8rTCpdGMtVZT311OLpUEO2mf5q5N9E-y5ZHq7K5TRXE-7QYKZ5nT/file# [following]
--2023-11-27 22:54:54--  https://ucd9707fc04aedc20b1343e1cc89.dl.dropboxusercontent.com/cd/0/inline/CIWkZzcNnFUKXAONc6AUfd

In [7]:
train_path = "./train_corpus_descriptions_airbnb.csv"
test_path = "./test_corpus_descriptions_airbnb.csv"
# Read, then decode for py2 compat.
corpus_size=25000
airbnb_reviews = pd.read_csv(train_path, header=None, names=["review"]).dropna().sample(corpus_size).reset_index(drop=True)
test_airbnb_reviews = pd.read_csv(test_path, header=None, names=["review"]).dropna().sample(2000).reset_index(drop=True)


In [8]:
airbnb_reviews

,review
0,"Comfortable, quiet garden-facing double room w..."
1,Welcome to the Giant Oaks House located just o...
2,The apartment is located just 2 minutes walk f...
3,Stay for 90+ nights (minimum nights and rates ...
4,"Ideal place for tourists , visiting los angele..."
...,...
24995,Stay in the charming historic California Heigh...
24996,"Bright, airy sun-drenched one-bedroom in the h..."
24997,Comfortable studio apartment in the middle of ...
24998,My place is less than 1/4 mile to the ship can...


In [9]:
# Take a look at the first review in text
print(airbnb_reviews.iloc[0].review)


Comfortable, quiet garden-facing double room with own en-suite (shower & toilet) on the 2nd floor of a lovely two-story 2.5 bedroom home, conveniently located in between 3 tubes stations, plenty of buses, bike hiring , walking distance to Hyde Park and canals. You are welcome to make yourself at home in all common areas including the large lounge, kitchen and communal garden as well as the study if you require a quiet work-desk.The spaceA large, bright, cozy home with extra high ceiling and full length double glazed windows, set over the ground and first floors. Comprising of a spacious carpeted lounge with leather sofas, flat screen TV, DVD player, streaming services, with large double doors opening to a comfortable eat- in kitchen with polished wooden floor and all mod cons. Free internet available on the wireless network which reaches all the way out to the quiet communal garden.The available tranquil bedroom looks out to the communal garde


In [10]:
import re
def preprocess_text(text, should_join=True):
    text = str(text)
    text = ' '.join(str(word).lower() for word in textblob_tokenizer(text))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [11]:
X_preprocessed = airbnb_reviews.review.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

In [12]:
def get_maximum_review_length(df):
    pre = df.swifter.apply(lambda x: preprocess_text(x, should_join=False))
    maximum = pre.swifter.apply(len).max()
    return maximum


maximum = get_maximum_review_length(airbnb_reviews.review)


Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

In [13]:
import itertools
from torchnlp.encoders import LabelEncoder


list_of_words = list(itertools.chain.from_iterable([preprocess_text(sentence, should_join=False) for sentence in airbnb_reviews.review]))
ids_from_words = LabelEncoder(list_of_words, reserved_labels=['UNK'], unknown_index=0, min_occurrences=3)


In [14]:

def ids_from_text(text):
  return ids_from_words.batch_encode(text)

def text_from_ids(ids):
  return ids_from_words.batch_decode(ids)


In [16]:
ids_from_text('Only you can prevent forest fires'.lower().split())

tensor([  134,    39,   291,  2416,  2430, 10637])

In [17]:
# Cool solution
def pad_sequence_of_tokens(x, maxlen, unk_token='[UNK]'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

def get_tensor(x, maximum=maximum):
  padding = (0, maximum-ids_from_text(x).shape[-1])
  return torch.squeeze(F.pad(ids_from_text(x), padding, "constant", 0).to(torch.long))


In [18]:
import torch.nn.functional as F
def get_ids_tensor(srs):

  processed = srs.swifter.apply(lambda x: pad_sequence_of_tokens(preprocess_text(x, should_join=False), maxlen=maximum))
  result = processed.swifter.apply(get_tensor).to_list()
  return torch.stack(result)

In [19]:
all_ids = get_ids_tensor(srs=X_preprocessed.reset_index(drop=True))
all_ids

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

tensor([[   1,    2,    3,  ...,    0,    0,    0],
        [  41,   34,   14,  ...,    0,    0,    0],
        [  14,  159,  140,  ...,    0,    0,    0],
        ...,
        [   1,  187,  159,  ...,    0,    0,    0],
        [ 520,  295,  140,  ...,    0,    0,    0],
        [2905,  237,   14,  ...,    0,    0,    0]])

In [32]:
class LSTMDataset(torch.utils.data.Dataset):
    def __init__(self, all_ids):
        self.all_ids = all_ids

    def __len__(self):
        return len(self.all_ids)

    def split_input_target(self, sequence):
        input_text = sequence[:-1]
        target_text = sequence[1:]
        return input_text, target_text

    def __getitem__(self, index):
        input_seq, target_seq = self.split_input_target(self.all_ids[index])
        return torch.tensor(input_seq), torch.tensor(target_seq)


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test= train_test_split(all_ids.numpy(), test_size=0.2, random_state=42)
X_train = torch.Tensor(X_train).to(torch.long)
X_test = torch.Tensor(X_test).to(torch.long)



In [38]:
train_ds = LSTMDataset(X_train)
test_ds = LSTMDataset(X_test)

In [54]:
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True, drop_last=True)
test_dl = DataLoader(test_ds, batch_size=256, shuffle=True, drop_last=True)

In [44]:
input_example, target_example = train_ds[0]
print("Input :", ' '.join(text_from_ids(input_example)))
print("Target:", ' '.join(text_from_ids(target_example)))

Input : bed bath stand alone beach cottage in sunset cliffs wrap around patio with ocean UNK updated UNK tub on the back UNK bbq driveway parking miles from the cliffs mile from the main beaches of ocean beach UNK porch daybed UNK built patio table close to shops bars restaurants comes with bikes surfboard boogie UNK in each UNK bedroom den not listed as bedroom with pull out single sleeper sq UNK spacesitting along hillside in the highly desired neighborhood of sunset cliffs the coral cottage offers endless bungalow style charm bright and calming coastal colors compliment the interior and capture the true essence of what beach cottage is all about the wrap around patio space offers ocean views and along with that view comes those sweet ocean breezes with no shortage on windows you ca UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK


In [55]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(in_features=hidden_dim, out_features=vocab_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        lstm_out, hidden = self.lstm(embedded, hidden)
        out = self.fc(lstm_out)
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().to(device),
            weight.new(self.num_layers, batch_size, self.hidden_dim).zero_().to(device)
        )
        return hidden


In [56]:
# Hyperparameters
sequence_length = 100  # length of sequence for a training example
embedding_dim = 256  # embedding dimension
hidden_dim = 512  # LSTM hidden dimensions
num_layers = 2  # number of LSTM layers
batch_size = 256  # batch size
num_epochs = 20  # number of epochs to train
lr = 0.001  # learning rate
vocab_size = len(ids_from_words.token_to_index)

In [57]:
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers).to(device)

In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [59]:
for epoch in range(num_epochs):
    model.train()
    hidden = model.init_hidden(batch_size)
    for batch, (x, y) in enumerate(train_dl):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred, hidden = model(x, hidden)
        loss = criterion(y_pred.transpose(1, 2), y)
        loss.backward()
        optimizer.step()
        for hidden_state in hidden:
          hidden_state.detach_()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 3.868939161300659
Epoch 2/20, Loss: 3.3676531314849854
Epoch 3/20, Loss: 3.1997694969177246
Epoch 4/20, Loss: 2.834071636199951
Epoch 5/20, Loss: 2.7280337810516357
Epoch 6/20, Loss: 2.608689785003662
Epoch 7/20, Loss: 2.634016513824463
Epoch 8/20, Loss: 2.554908037185669
Epoch 9/20, Loss: 2.4868693351745605
Epoch 10/20, Loss: 2.272606372833252
Epoch 11/20, Loss: 2.1695444583892822
Epoch 12/20, Loss: 2.2438242435455322
Epoch 13/20, Loss: 2.2575199604034424
Epoch 14/20, Loss: 2.1464107036590576
Epoch 15/20, Loss: 2.0029776096343994
Epoch 16/20, Loss: 2.104871988296509
Epoch 17/20, Loss: 2.128967761993408
Epoch 18/20, Loss: 2.0224568843841553
Epoch 19/20, Loss: 2.0458643436431885
Epoch 20/20, Loss: 1.9865281581878662


In [64]:
# Function to generate text from the model
def generate_text(model, start_string, generation_length=1000):
    model.eval()  # evaluation mode

    # Begin with the start string, and predict the next word for generation_length steps
    words = start_string.split()
    hidden = model.init_hidden(1)
    for _ in range(generation_length):
        indexes = []
        for word in words:
          if word in ids_from_words.token_to_index:
            index = ids_from_words.token_to_index[word]
          else:
            index = 0
          indexes.append(index)
        x = torch.tensor([indexes], dtype=torch.long).to(device)
        out, hidden = model(x, hidden)

        out_dist = torch.exp(out[:, -1])
        top_i = torch.multinomial(out_dist, 1)[0]

        words.append(ids_from_words.index_to_token[top_i.item()])

    return ' '.join(words)

In [65]:
import time
start = time.time()
start_string = 'I was walking down the pathway'

result = generate_text(model, start_string)

end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)



I was walking down the pathway to collect guest acc leave the house is limited though it does not throw responsible for rest and people there also direct wall door brings the guest house during their stay please contact me as cancel with any questions needs other things to notewe have new hgtv occupants you can use your party thank you for work all UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK U